In [1]:
conda install -c conda-forge mlxtend=0.23.1


Retrieving notices: ...working... done
Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - mlxtend=0.23.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.12.14 |       hf0a4a13_0         153 KB  conda-forge
    certifi-2024.12.14         |     pyhd8ed1ab_0         158 KB  conda-forge
    conda-24.11.3              |  py312h81bd7bf_0         1.1 MB  conda-forge
    contourpy-1.3.1            |  py312h48ca7d4_0         264 KB
    cycler-0.12.1              |     pyhd8ed1ab_1          13 KB  conda-forge
    fonttools-4.55.3           |  py312h998013c_1         2.6 MB  conda-forge
    libexpat-2.6.2             |       hebf3989_0          62 KB  conda-forge
    libsqlite-3.46.0           |       hfb93653_0         811 KB  conda-forge
    libzlib-1.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sklearn.preprocessing import MinMaxScaler
import datetime as dt
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
sns.set_theme(style="whitegrid")  # Tema do seaborn

In [2]:
# Load data
reviews_attractions = pd.read_excel("Reviews_Attractions_Final.xlsx")
reviews_attractions.head()

,localID,globalRating,positionOnRanking,totalReviews,userName,userContributions,tripType,reviewVisited,reviewRating,userCountryUntrieted,timeDifferenceDays,visitedDuringHoliday,seasonVisited,Name,Country,ISO
0,MAG001,4.5,1,163828,Maurice M@MauriceM171,29,Couples,2021-08-01,5,In,17,1,Summer,Basilica of the Sagrada Familia,SPAIN,ES
1,MAG001,4.5,1,163828,nickchamb@nickchamb,7,Couples,2021-03-01,5,Ireland,169,0,Winter,Basilica of the Sagrada Familia,SPAIN,ES
2,MAG001,4.5,1,163828,Arthur Villon@arthur4588,268,Friends,2021-08-01,5,Spain,15,1,Summer,Basilica of the Sagrada Familia,SPAIN,ES
3,MAG001,4.5,1,163828,James U@696jamesu,7,Unkown,2021-07-01,5,Unkown,46,1,Summer,Basilica of the Sagrada Familia,SPAIN,ES
4,MAG001,4.5,1,163828,Kamal K Sirra@Sirra63,516,Family,2021-07-01,4,Spain,45,1,Summer,Basilica of the Sagrada Familia,SPAIN,ES


In [3]:
reviews_attractions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82757 entries, 0 to 82756
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   localID               82757 non-null  object        
 1   globalRating          82757 non-null  float64       
 2   positionOnRanking     82757 non-null  int64         
 3   totalReviews          82757 non-null  int64         
 4   userName              82757 non-null  object        
 5   userContributions     82757 non-null  int64         
 6   tripType              82757 non-null  object        
 7   reviewVisited         82757 non-null  datetime64[ns]
 8   reviewRating          82757 non-null  int64         
 9   userCountryUntrieted  82757 non-null  object        
 10  timeDifferenceDays    82757 non-null  int64         
 11  visitedDuringHoliday  82757 non-null  int64         
 12  seasonVisited         82757 non-null  object        
 13  Name            

In [4]:
# Create a pivot table
pt = pd.pivot_table(reviews_attractions[['userName','localID']],
                    index='userName',                                  # Each row will represent a traveler (trip)
                    columns='localID',                                 # Each attraction will be a column (tourist site)
                    aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)   # The value should be 1 if the attraction is present in the trip, otherwise 0
pt.head()

localID,MAG001,MAG002,MAG003,MAG004,MAG005,MAG006,MAG007,MAG008,MAG009,MAG010,MAG011,MAG012,MAG013,MAG014,MAG015,MAG016,MAG017,MAG018,MAG019,MAG020,MAG021,MAG022,MAG023,MAG024,MAG025,MAG026,MAG027,MAG028,MAG029,MAG030,MAG031,MAG032,MAG033,MAG034,MAG035,MAG036,MAG037,MAG038,MAG039,MAG040,MAG041,MAG042,MAG043,MAG044,MAG045,MAG046,MAG047,MAG048,MAG049,MAG050,MAG051,MAG052,MAG053,MAG054,MAG055,MAG056,MAG057,MAG058,MAG059,MAG060,MAG061,MAG062,MAG063,MAG064,MAG065,MAG066,MAG067,MAG068,MAG069,MAG070,MAG071,MAG072,MAG073,MAG074,MAG075,MAG076,MAG077,MAG078,MAG079,MAG080,MAG081,MAG082,MAG083,MAG084,MAG085,MAG086,MAG087,MAG088,MAG089,MAG090,MAG091,MAG092,MAG093,MAG094,MAG095,MAG096,MAG097,MAG098,MAG099,MAG100
userName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
#1 Best@Number1Best,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#Ciccone@932emzc,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#Travelendlessbeauty@travelendlessbeauty,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#moving mountains@peaceat28,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#taxi de Barcelona #taxi#aeropuer@cavataxi,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Converting DataFrame to boolean
pt = pt.astype(bool)

# association rule mining
frequent_itemsets = apriori(pt, min_support=0.0001, use_colnames=True)

In [6]:
# Association rules by support
rulesSupport = association_rules(frequent_itemsets, metric="support", min_threshold=0.005)
rulesSupport.sort_values(by='support', ascending=False, inplace=True)
rulesSupport.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(MAG002),(MAG001),0.041063,0.119783,0.023532,0.573059,4.784141,0.018613,2.061684,0.824847
1,(MAG001),(MAG002),0.119783,0.041063,0.023532,0.196452,4.784141,0.018613,1.193378,0.898615
2,(MAG001),(MAG004),0.119783,0.025047,0.012578,0.105009,4.192439,0.009578,1.089344,0.865100
3,(MAG004),(MAG001),0.025047,0.119783,0.012578,0.502183,4.192439,0.009578,1.768155,0.781038
17,(MAG019),(MAG012),0.018672,0.028360,0.009906,0.530544,18.707593,0.009377,2.069715,0.964556
16,(MAG012),(MAG019),0.028360,0.018672,0.009906,0.349311,18.707593,0.009377,1.508137,0.974173
4,(MAG023),(MAG001),0.014172,0.119783,0.008766,0.618523,5.163687,0.007068,2.307389,0.817932
5,(MAG001),(MAG023),0.119783,0.014172,0.008766,0.073180,5.163687,0.007068,1.063667,0.916070
10,(MAG022),(MAG003),0.024141,0.075548,0.007906,0.327508,4.335096,0.006083,1.374666,0.788356
11,(MAG003),(MAG022),0.075548,0.024141,0.007906,0.104654,4.335096,0.006083,1.089923,0.832195


In [12]:
frequent_itemsets[ frequent_itemsets['itemsets'] == {'MAG010'}]

,support,itemsets
9,0.01986,(MAG010)


The top 10 association rules displayed above highlight the most significant and insightful relationships within the context of the analyzed tourist attractions. These metrics provide valuable insights into patterns and preferences related to these sites.

Support: This metric indicates the proportion of transactions or records in the dataset that include both the antecedent and the consequent of a rule. For example, a support value of 0.023532 means that about 2.35% of all transactions contain both components of the rule.

Confidence: Confidence reflects the likelihood of the consequent appearing when the antecedent is present. For instance, if the confidence is 0.573059, it implies that approximately 57.64% of the cases where the antecedent occurs also include the consequent.

Lift: Lift evaluates how much more likely the consequent is to occur when the antecedent is present, compared to its general likelihood of occurrence. A lift greater than 1 suggests a positive relationship, where the antecedent increases the probability of the consequent. Conversely, a lift less than 1 indicates a negative association. For example, a lift of 4.784141 shows a significant increase in the likelihood of the consequent when the antecedent is present.

In [8]:
# association rules by confidence
rulesConfidence = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.20)
rulesConfidence.sort_values(by='confidence', ascending=False, inplace=True)
rulesConfidence.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
3390,"(MAG008, MAG002, MAG012, MAG030)",(MAG019),0.000109,0.018672,0.000109,1.0,53.555649,0.000107,inf,0.981435
4037,"(MAG023, MAG015, MAG019, MAG001, MAG004)",(MAG002),0.000109,0.041063,0.000109,1.0,24.352740,0.000105,inf,0.959042
2804,"(MAG023, MAG030, MAG012, MAG001)",(MAG002),0.000109,0.041063,0.000109,1.0,24.352740,0.000105,inf,0.959042
1452,"(MAG059, MAG012, MAG001)",(MAG019),0.000172,0.018672,0.000172,1.0,53.555649,0.000169,inf,0.981497
4321,"(MAG002, MAG012, MAG015, MAG030, MAG004)",(MAG019),0.000109,0.018672,0.000109,1.0,53.555649,0.000107,inf,0.981435
4074,"(MAG023, MAG030, MAG019, MAG001, MAG004)",(MAG002),0.000109,0.041063,0.000109,1.0,24.352740,0.000105,inf,0.959042
1261,"(MAG023, MAG014, MAG004)",(MAG001),0.000266,0.119783,0.000266,1.0,8.348422,0.000234,inf,0.880451
4069,"(MAG002, MAG023, MAG030, MAG019, MAG001)",(MAG004),0.000109,0.025047,0.000109,1.0,39.924517,0.000107,inf,0.975059
2584,"(MAG015, MAG030, MAG001, MAG004)",(MAG002),0.000109,0.041063,0.000109,1.0,24.352740,0.000105,inf,0.959042
91,"(MAG002, MAG027)",(MAG001),0.000188,0.119783,0.000188,1.0,8.348422,0.000165,inf,0.880382


The provided code creates association rules using the confidence metric and organizes them based on their Confidence scores. As an example, the first rule in the output highlights a relationship where the antecedent includes MAG022, MAG002, and MAG001. This rule demonstrates a confidence of 1.0, signifying that whenever the antecedent occurs, the consequent is guaranteed to follow. Furthermore, the lift value of 39.162465 emphasizes a highly significant connection between the antecedent and the consequent.

(MAG022, MAG002) -> (MAG001): 
Take note of the first row, where Westminster Abbey(England), Parc Guell (Spain), form the antecedent for the Parc Guel (Spain). It is important to highlight that the Confidence level for this rule is 1.0, indicating that individuals who visited this highly frequented tourist site in the dataset also probably will visit Parc Guel.


As observed, the supports for all the itemsets are notably low. However, despite the fact that few people visit both locations, those who visit one have a high probability of visiting the other, as indicated by the high confidence and lift values.

In [10]:
#Generate the association rules - by lift
rulesLift = association_rules(frequent_itemsets, metric="lift", min_threshold=1.5)
rulesLift.sort_values(by='lift', ascending=False, inplace=True)
rulesLift.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
5764,"(MAG010, MAG058)","(MAG089, MAG014)",0.000234,0.000125,0.000109,0.466667,3733.275000,0.000109,1.874766,0.999967
5769,"(MAG089, MAG014)","(MAG010, MAG058)",0.000125,0.000234,0.000109,0.875000,3733.275000,0.000109,7.998125,0.999857
5765,"(MAG010, MAG089)","(MAG058, MAG014)",0.000156,0.000203,0.000109,0.700000,3446.100000,0.000109,3.332656,0.999866
5768,"(MAG058, MAG014)","(MAG010, MAG089)",0.000203,0.000156,0.000109,0.538462,3446.100000,0.000109,2.166328,0.999913
10082,"(MAG012, MAG008, MAG002)","(MAG019, MAG015, MAG030)",0.000250,0.000156,0.000109,0.437500,2799.956250,0.000109,1.777500,0.999893
10099,"(MAG019, MAG015, MAG030)","(MAG012, MAG008, MAG002)",0.000156,0.000250,0.000109,0.700000,2799.956250,0.000109,3.332500,0.999799
10090,"(MAG012, MAG015, MAG030)","(MAG008, MAG019, MAG002)",0.000141,0.000281,0.000109,0.777778,2765.388889,0.000109,4.498734,0.999779
10091,"(MAG008, MAG019, MAG002)","(MAG012, MAG015, MAG030)",0.000281,0.000141,0.000109,0.388889,2765.388889,0.000109,1.636134,0.999920
9974,"(MAG019, MAG030, MAG002)","(MAG012, MAG004, MAG015)",0.000172,0.000234,0.000109,0.636364,2715.109091,0.000109,2.749355,0.999804
9959,"(MAG012, MAG004, MAG015)","(MAG019, MAG030, MAG002)",0.000234,0.000172,0.000109,0.466667,2715.109091,0.000109,1.874678,0.999866


In [14]:
#Add a column with the length
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

In [22]:
# Length=2 and Support>=0.1
frequent_itemsets[(frequent_itemsets['length'] == 2) & (frequent_itemsets['support'] >= 0.0080)]

,support,itemsets,length
100,0.023532,"(MAG002, MAG001)",2
102,0.012578,"(MAG001, MAG004)",2
121,0.008766,"(MAG023, MAG001)",2
509,0.009906,"(MAG012, MAG019)",2


In [27]:
# High Confidence and high Lift
rulesConfidence[(rulesConfidence['confidence'] >= 1) & (rulesConfidence['lift'] >= 500)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4337,"(MAG015, MAG012, MAG030, MAG004)","(MAG002, MAG019)",0.000109,0.000719,0.000109,1.0,1391.282609,0.000109,inf,0.999391
4284,"(MAG023, MAG002, MAG012, MAG015)","(MAG019, MAG004)",0.000109,0.000563,0.000109,1.0,1777.750000,0.000109,inf,0.999547
4340,"(MAG015, MAG030, MAG019, MAG004)","(MAG002, MAG012)",0.000109,0.000906,0.000109,1.0,1103.431034,0.000109,inf,0.999203


In [35]:
# High Confidence rules where 'MAG010' is in the RHS
rulesConfidence[['MAG010' in elem for elem in rulesConfidence['consequents']]]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1410,"(MAG014, MAG058, MAG001)",(MAG010),0.000109,0.019860,0.000109,1.000000,50.353265,0.000107,inf,0.980248
672,"(MAG098, MAG014)",(MAG010),0.000125,0.019860,0.000125,1.000000,50.353265,0.000123,inf,0.980263
662,"(MAG014, MAG058)",(MAG010),0.000203,0.019860,0.000188,0.923077,46.479937,0.000183,12.741824,0.978684
656,"(MAG014, MAG042)",(MAG010),0.000141,0.019860,0.000125,0.888889,44.758458,0.000122,8.821263,0.977795
2154,"(MAG089, MAG014)","(MAG058, MAG010)",0.000125,0.000234,0.000109,0.875000,3733.275000,0.000109,7.998125,0.999857
...,...,...,...,...,...,...,...,...,...,...
3616,"(MAG014, MAG019)","(MAG008, MAG012, MAG010)",0.000578,0.000156,0.000125,0.216216,1383.762162,0.000125,1.275663,0.999855
3680,"(MAG014, MAG019)","(MAG032, MAG012, MAG010)",0.000578,0.000234,0.000125,0.216216,922.508108,0.000125,1.275563,0.999494
2142,"(MAG014, MAG032)","(MAG047, MAG010)",0.002703,0.001422,0.000578,0.213873,150.413708,0.000574,1.270250,0.996044
3634,"(MAG014, MAG008)","(MAG015, MAG019, MAG010)",0.000516,0.000188,0.000109,0.212121,1131.295455,0.000109,1.268993,0.999632


In [36]:
# High Confidence rules where 'MAG014' is in the RHS
rulesConfidence[['MAG014' in elem for elem in rulesConfidence['consequents']]]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
3599,"(MAG008, MAG012, MAG019, MAG010)",(MAG014),0.000125,0.016844,0.000125,1.000000,59.368275,0.000123,inf,0.983279
1976,"(MAG008, MAG019, MAG010)",(MAG014),0.000203,0.016844,0.000188,0.923077,54.801484,0.000184,12.781028,0.981952
1949,"(MAG008, MAG012, MAG010)",(MAG014),0.000156,0.016844,0.000141,0.900000,53.431447,0.000138,9.831560,0.981438
2078,"(MAG015, MAG012, MAG010)",(MAG014),0.000156,0.016844,0.000141,0.900000,53.431447,0.000138,9.831560,0.981438
673,"(MAG098, MAG010)",(MAG014),0.000141,0.016844,0.000125,0.888889,52.771800,0.000123,8.848404,0.981188
...,...,...,...,...,...,...,...,...,...,...
2143,"(MAG032, MAG010)","(MAG047, MAG014)",0.002703,0.001297,0.000578,0.213873,164.911414,0.000575,1.270409,0.996630
3614,"(MAG019, MAG010)","(MAG014, MAG008, MAG012)",0.000594,0.000234,0.000125,0.210526,898.231579,0.000125,1.266370,0.999480
3679,"(MAG019, MAG010)","(MAG014, MAG032, MAG012)",0.000594,0.000203,0.000125,0.210526,1036.421053,0.000125,1.266409,0.999629
3676,"(MAG012, MAG010)","(MAG014, MAG032, MAG019)",0.000609,0.000203,0.000125,0.205128,1009.846154,0.000125,1.257809,0.999619


In [37]:
# High Confidence rules where 'MAG021' is in the RHS
rulesConfidence[['MAG021' in elem for elem in rulesConfidence['consequents']]]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
856,"(MAG052, MAG049)",(MAG021),0.000141,0.012891,0.000125,0.888889,68.955152,0.000123,8.883983,0.985636
2229,"(MAG047, MAG014, MAG049)",(MAG021),0.000141,0.012891,0.000109,0.777778,60.335758,0.000108,4.441991,0.983564
2335,"(MAG047, MAG032, MAG049)",(MAG021),0.000188,0.012891,0.000141,0.750000,58.180909,0.000138,3.948437,0.982997
854,"(MAG047, MAG049)",(MAG021),0.000250,0.012891,0.000188,0.750000,58.180909,0.000184,3.948437,0.983058
2131,"(MAG014, MAG049, MAG010)",(MAG021),0.000391,0.012891,0.000281,0.720000,55.853673,0.000276,3.525390,0.982480
3683,"(MAG014, MAG049, MAG032, MAG010)",(MAG021),0.000219,0.012891,0.000156,0.714286,55.410390,0.000153,3.454882,0.982168
2221,"(MAG014, MAG032, MAG049)",(MAG021),0.000313,0.012891,0.000219,0.700000,54.302182,0.000215,3.290364,0.981891
2165,"(MAG049, MAG032, MAG010)",(MAG021),0.000297,0.012891,0.000203,0.684211,53.077321,0.000199,3.125846,0.981451
755,"(MAG014, MAG049)",(MAG021),0.000625,0.012891,0.000406,0.650000,50.423455,0.000398,2.820312,0.980781
852,"(MAG032, MAG049)",(MAG021),0.000578,0.012891,0.000375,0.648649,50.318624,0.000368,2.809465,0.980694


In [38]:
# High Confidence rules where 'MAG032' is in the RHS
rulesConfidence[['MAG032' in elem for elem in rulesConfidence['consequents']]]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2169,"(MAG047, MAG049, MAG010)",(MAG032),0.000125,0.015469,0.000125,1.000000,64.645455,0.000123,inf,0.984654
739,"(MAG047, MAG012)",(MAG032),0.000109,0.015469,0.000109,1.000000,64.645455,0.000108,inf,0.984639
2232,"(MAG047, MAG014, MAG049)",(MAG032),0.000141,0.015469,0.000125,0.888889,57.462626,0.000123,8.860779,0.982736
845,"(MAG047, MAG019)",(MAG032),0.000125,0.015469,0.000109,0.875000,56.564773,0.000107,7.876248,0.982444
2214,"(MAG021, MAG047, MAG014)",(MAG032),0.000172,0.015469,0.000141,0.818182,52.891736,0.000138,5.414921,0.981262
...,...,...,...,...,...,...,...,...,...,...
213,"(MAG001, MAG010)",(MAG032),0.001016,0.015469,0.000219,0.215385,13.923636,0.000203,1.254794,0.929123
2003,"(MAG014, MAG008)","(MAG032, MAG012)",0.000516,0.000484,0.000109,0.212121,437.920821,0.000109,1.268616,0.998231
589,"(MAG014, MAG008)",(MAG032),0.000516,0.015469,0.000109,0.212121,13.712672,0.000101,1.249597,0.927553
3679,"(MAG019, MAG010)","(MAG014, MAG032, MAG012)",0.000594,0.000203,0.000125,0.210526,1036.421053,0.000125,1.266409,0.999629


In [39]:
# High Confidence rules where 'MAG047' is in the RHS
rulesConfidence[['MAG047' in elem for elem in rulesConfidence['consequents']]]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
475,"(MAG032, MAG004)",(MAG047),0.000234,0.008985,0.000109,0.466667,51.941217,0.000107,1.858154,0.980977
2171,"(MAG049, MAG032, MAG010)",(MAG047),0.000297,0.008985,0.000125,0.421053,46.864256,0.000122,1.711754,0.978952
2234,"(MAG014, MAG032, MAG049)",(MAG047),0.000313,0.008985,0.000125,0.400000,44.521043,0.000122,1.651692,0.977844
2334,"(MAG021, MAG032, MAG049)",(MAG047),0.000375,0.008985,0.000141,0.375000,41.738478,0.000137,1.585625,0.976407
2139,"(MAG014, MAG032, MAG010)",(MAG047),0.001547,0.008985,0.000578,0.373737,41.597945,0.000564,1.582428,0.977472
758,"(MAG014, MAG032)",(MAG047),0.002703,0.008985,0.001000,0.369942,41.175532,0.000976,1.572896,0.978358
2216,"(MAG021, MAG014, MAG032)",(MAG047),0.000391,0.008985,0.000141,0.360000,40.068939,0.000137,1.548462,0.975424
2160,"(MAG021, MAG032, MAG010)",(MAG047),0.000391,0.008985,0.000141,0.360000,40.068939,0.000137,1.548462,0.975424
273,"(MAG032, MAG001)",(MAG047),0.000703,0.008985,0.000250,0.355556,39.574261,0.000244,1.537783,0.975417
687,"(MAG032, MAG010)",(MAG047),0.002703,0.008985,0.000953,0.352601,39.245428,0.000929,1.530765,0.977161


In [40]:
# High Confidence rules where 'MAG049' is in the RHS
rulesConfidence[['MAG049' in elem for elem in rulesConfidence['consequents']]]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2226,"(MAG021, MAG047, MAG014)",(MAG049),0.000172,0.004672,0.000109,0.636364,136.209486,0.000109,2.737152,0.992829
3681,"(MAG021, MAG014, MAG032, MAG010)",(MAG049),0.000266,0.004672,0.000156,0.588235,125.907928,0.000155,2.417225,0.992321
855,"(MAG021, MAG052)",(MAG049),0.000219,0.004672,0.000125,0.571429,122.310559,0.000124,2.322432,0.992041
2218,"(MAG021, MAG014, MAG032)",(MAG049),0.000391,0.004672,0.000219,0.560000,119.864348,0.000217,2.262109,0.992045
2332,"(MAG021, MAG047, MAG032)",(MAG049),0.000266,0.004672,0.000141,0.529412,113.317136,0.000139,2.115072,0.991439
2162,"(MAG021, MAG032, MAG010)",(MAG049),0.000391,0.004672,0.000203,0.520000,111.302609,0.000201,2.073600,0.991403
697,"(MAG093, MAG010)",(MAG049),0.000234,0.004672,0.000109,0.466667,99.886957,0.000108,1.866240,0.990221
853,"(MAG021, MAG047)",(MAG049),0.000406,0.004672,0.000188,0.461538,98.789298,0.000186,1.848466,0.990280
2128,"(MAG021, MAG014, MAG010)",(MAG049),0.000609,0.004672,0.000281,0.461538,98.789298,0.000278,1.848466,0.990481
851,"(MAG021, MAG032)",(MAG049),0.000828,0.004672,0.000375,0.452830,96.925349,0.000371,1.819048,0.990503


In [41]:
# High Confidence rules where 'MAG093' is in the RHS
rulesConfidence[['MAG093' in elem for elem in rulesConfidence['consequents']]]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


Focusing on the Portuguese attractions to uncover insights beneficial for Portugal's tourism, we observe that the supports for all itemsets are notably low. Nevertheless, despite the fact that few visitors explore both locations, those who visit one are highly likely to visit the other, as evidenced by the high confidence and lift values.